In [1]:
#imports
import pandas as pd
import os
import re

In [2]:
base_path = os.getcwd()
onet_db_path = os.path.join(base_path, "ONet")
output_path = os.path.join(base_path, "ONet_Output")
def sanitize_filename(name):
    """
    Removes special characters from a string to make it a valid filename.
    """
    # Remove characters that are not letters, numbers, underscores, or hyphens
    name = re.sub(r'[^\w\s-]', '', name)
    # Replace whitespace and hyphens with a single underscore
    name = re.sub(r'[-\s]+', '_', name)
    return name


In [3]:
def process_onet_data_basic(onet_db_path, output_path):
    """
    Reads O*NET data from Excel files, processes it, and generates a .txt file
    for each occupation with its description and tasks.

    Args:
        onet_db_path (str): The path to the directory containing O*NET .xlsx files.
        output_path (str): The directory where the .txt files will be saved.
    """
    print("Starting O*NET data processing...")

    # --- 1. Create Output Directory ---
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print(f"Created output directory: {output_path}")

    # --- 2. Define File Paths ---
    # These are the key files we need for descriptions and tasks.
    try:
        occupation_data_file = os.path.join(onet_db_path, "Occupation Data.xlsx")
        tasks_file = os.path.join(onet_db_path, "Task Statements.xlsx")

        # --- 3. Load Data into Pandas DataFrames ---
        print("Loading data from Excel files...")
        df_occupations = pd.read_excel(occupation_data_file)
        df_tasks = pd.read_excel(tasks_file)
        print("Data loaded successfully.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please ensure the O*NET database files are in the specified directory.")
        return

    # --- 4. Process Each Occupation ---
    print(f"Found {len(df_occupations)} occupations. Generating files...")

    # Iterate over each row in the occupations DataFrame. Each row is one job.
    for index, row in df_occupations.iterrows():
        # Get the unique code and title for the current occupation
        soc_code = row['O*NET-SOC Code']
        title = row['Title']
        description = row['Description']

        # --- 5. Find All Tasks for the Current Occupation ---
        # Filter the tasks DataFrame to get only the tasks for the current soc_code
        occupation_tasks = df_tasks[df_tasks['O*NET-SOC Code'] == soc_code]

        # --- 6. Generate the .txt File Content ---
        # We use f-strings to build the content for our file.
        # This is where you will add more data categories later.
        file_content = []
        file_content.append(f"Job Title: {title}")
        file_content.append(f"O*NET-SOC Code: {soc_code}\n")

        file_content.append("# DESCRIPTION")
        file_content.append(f"{description}\n")

        file_content.append("# TASKS")
        if not occupation_tasks.empty:
            for task_index, task_row in occupation_tasks.iterrows():
                file_content.append(f"- {task_row['Task']}")
        else:
            file_content.append("No specific tasks listed for this occupation.")

        # --- 7. Save the .txt File ---
        # Sanitize the title to create a safe filename (e.g., "Software_Developers")
        sanitized_title = sanitize_filename(title)
        output_filename = f"{sanitized_title}_{soc_code}.txt"
        output_filepath = os.path.join(output_path, output_filename)

        # Join the content list into a single string with newlines and write to the file
        with open(output_filepath, 'w', encoding='utf-8') as f:
            f.write('\n'.join(file_content))

    print(f"\nProcessing complete. {len(df_occupations)} files were generated in '{output_path}'.")


In [4]:
process_onet_data_basic(onet_db_path, output_path)

Starting O*NET data processing...
Created output directory: /Users/leozhu/Documents/UBC-CIC-Hackathon-2025/data/ONet_Output
Loading data from Excel files...
Data loaded successfully.
Found 1016 occupations. Generating files...

Processing complete. 1016 files were generated in '/Users/leozhu/Documents/UBC-CIC-Hackathon-2025/data/ONet_Output'.


In [8]:
def add_knowledge(onet_db_path, output_path):
    """
    Appends Knowledge data to the existing occupation .txt files.

    Args:
        onet_db_path (str): The path to the directory containing O*NET .xlsx files.
        output_path (str): The directory where the .txt files are located.
    """
    print("\nStarting to add Knowledge data...")

    # --- 1. Load Data ---
    try:
        occupation_data_file = os.path.join(onet_db_path, "Occupation Data.xlsx")
        knowledge_file = os.path.join(onet_db_path, "Knowledge.xlsx")

        print("Loading knowledge data...")
        df_occupations = pd.read_excel(occupation_data_file)
        df_knowledge = pd.read_excel(knowledge_file)
        print("Data loaded.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Could not find a required Excel file.")
        return

    # --- 2. Process Each Occupation ---
    print(f"Enriching {len(df_occupations)} files with knowledge data...")
    for index, row in df_occupations.iterrows():
        soc_code = row['O*NET-SOC Code']
        title = row['Title']

        # Recreate the filename
        sanitized_title = sanitize_filename(title)
        output_filename = f"{sanitized_title}_{soc_code}.txt"
        output_filepath = os.path.join(output_path, output_filename)

        if not os.path.exists(output_filepath):
            continue

        # --- 3. Filter and Format Knowledge Data ---
        job_knowledge = (
            df_knowledge[df_knowledge['O*NET-SOC Code'] == soc_code]
            .pivot_table(
                index=['Element Name'],
                columns='Scale Name',
                values='Data Value'
            )
            .reset_index()
            .fillna(0)
        )
        knowledge_content = ["\n\n# KNOWLEDGE"]
        # Add context for the LLM
        knowledge_content.append("(Scale definitions: Importance is rated 1-5. Level is rated 0-7.)")

        if not job_knowledge.empty:
            # Sort by importance
            job_knowledge = job_knowledge.sort_values(by='Importance', ascending=False)
            for _, knowledge_row in job_knowledge.iterrows():
                name = knowledge_row['Element Name']
                importance = knowledge_row['Importance']
                level = knowledge_row['Level']
                knowledge_content.append(f"- {name} (Importance: {importance}/5 | Level: {level}/7)")
        else:
            knowledge_content.append("- No specific knowledge areas listed.")

        # --- 4. Append to File ---
        with open(output_filepath, 'a', encoding='utf-8') as f:
            f.write('\n'.join(knowledge_content))

    print("Successfully added Knowledge data to all files.")


In [9]:
add_knowledge(onet_db_path, output_path)


Starting to add Knowledge data...
Loading knowledge data...
Data loaded.
Enriching 1016 files with knowledge data...
Successfully added Knowledge data to all files.


In [10]:
def add_skills_and_tech(onet_db_path, output_path):
    """
    Appends Skills and Technology Tools data to the existing occupation .txt files.

    Args:
        onet_db_path (str): The path to the directory containing O*NET .xlsx files.
        output_path (str): The directory where the .txt files are located.
    """
    print("\nStarting to add Skills and Technology data...")

    # --- 1. Define File Paths and Load Data ---
    try:
        # We need Occupation Data again to map SOC Codes to the filenames
        occupation_data_file = os.path.join(onet_db_path, "Occupation Data.xlsx")
        skills_file = os.path.join(onet_db_path, "Skills.xlsx")
        tech_file = os.path.join(onet_db_path, "Technology Skills.xlsx")
        tool_file = os.path.join(onet_db_path, "Tools Used.xlsx")

        print("Loading additional data...")
        df_occupations = pd.read_excel(occupation_data_file)
        df_skills = pd.read_excel(skills_file)
        df_tech = pd.read_excel(tech_file)
        df_tools = pd.read_excel(tool_file)
        print("Data loaded.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Could not find a required Excel file.")
        return

    # --- 2. Process Each Occupation and Append to its File ---
    print(f"Enriching {len(df_occupations)} files...")
    for index, row in df_occupations.iterrows():
        soc_code = row['O*NET-SOC Code']
        title = row['Title']

        # Recreate the exact filename from the first step
        sanitized_title = sanitize_filename(title)
        output_filename = f"{sanitized_title}_{soc_code}.txt"
        output_filepath = os.path.join(output_path, output_filename)

        if not os.path.exists(output_filepath):
            continue # Skip if for some reason the base file doesn't exist

        # --- 3. Filter and Format Skills Data ---
        # Get all skills for the current job
        job_skills = (
            df_skills[df_skills['O*NET-SOC Code'] == soc_code]
            .pivot_table(
                index=['Element Name'],
                columns='Scale Name',
                values='Data Value'
            )
            .reset_index()
            .fillna(0)
        )

        skills_content = ["\n\n# SKILLS", "(Scale definitions: Importance is rated 1-5. Level is rated 0-7.)"]

        if not job_skills.empty:
            # Sort skills by importance to show the most relevant ones first
            job_skills = job_skills.sort_values(by='Importance', ascending=False)
            for _, skill_row in job_skills.iterrows():
                skill_name = skill_row['Element Name']
                importance = skill_row['Importance']
                level = skill_row['Level']
                skills_content.append(f"- {skill_name} (Importance: {importance}/5 | Level: {level}/7)")
        else:
            skills_content.append("- No specific skills listed.")


        # --- 4. Filter and Format Technology Data ---
        job_tech = df_tech[df_tech['O*NET-SOC Code'] == soc_code]

        tech_content = ["\n\n# TECHNOLOGY SKILLS & TOOLS"]
        if not job_tech.empty:
            # Group technologies by their category for cleaner output
            for _, tech_skill_row in job_tech.iterrows():
                tech_skill = tech_skill_row["Example"]
                commodity_title = tech_skill_row["Commodity Title"]
                tech_content.append(f"- {tech_skill} (Tech skill type: {commodity_title})")
        else:
            tech_content.append("- No specific technology listed.")

        tool_content = df_tools[df_tools['O*NET-SOC Code'] == soc_code]
        if not tool_content.empty:
            tech_content.append("\n## TOOLS USED")
            for _, tool_row in tool_content.iterrows():
                tool_name = tool_row['Example']
                commodity_title = tool_row['Commodity Title']
                tech_content.append(f"- {tool_name} (Tool type: {commodity_title})")
        else:
            tech_content.append("- No specific tools listed.")


        # --- 5. Append All New Content to the Existing File ---
        # We open the file in 'a' (append) mode
        with open(output_filepath, 'a', encoding='utf-8') as f:
            f.write('\n'.join(skills_content))
            f.write('\n'.join(tech_content))

    print("Successfully added Skills and Technology data to all files.")


In [11]:
add_skills_and_tech(onet_db_path, output_path)


Starting to add Skills and Technology data...
Loading additional data...
Data loaded.
Enriching 1016 files...
Successfully added Skills and Technology data to all files.


In [12]:
def add_context_and_interests(onet_db_path, output_path):
    """
    Appends Work Context and Interests data to the existing occupation .txt files.
    This function handles the complexity of merging context categories.

    Args:
        onet_db_path (str): The path to the directory containing O*NET .xlsx files.
        output_path (str): The directory where the .txt files are located.
    """
    print("\nStarting to add Work Context and Interests data...")

    # --- 1. Define File Paths and Load Data ---
    try:
        occupation_data_file = os.path.join(onet_db_path, "Occupation Data.xlsx")
        work_context_file = os.path.join(onet_db_path, "Work Context.xlsx")
        context_categories_file = os.path.join(onet_db_path, "Work Context Categories.xlsx")
        interests_file = os.path.join(onet_db_path, "Interests.xlsx")

        print("Loading context and interests data...")
        df_occupations = pd.read_excel(occupation_data_file)
        df_work_context = pd.read_excel(work_context_file)
        df_context_categories = pd.read_excel(context_categories_file)
        df_interests = pd.read_excel(interests_file)
        print("Data loaded.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Could not find a required Excel file.")
        return

    # --- 2. Pre-process and Merge Work Context Data ---
    # We only care about the categorical context data, which has the 'CXP' Scale ID.
    df_context_categorical = df_work_context[df_work_context['Scale ID'] == 'CXP'].copy()

    # Merge with the category descriptions to get human-readable text
    # We need to rename the category column in the lookup table to avoid conflicts
    df_merged_context = pd.merge(
        df_context_categorical,
        df_context_categories.rename(columns={'Category': 'Category Code'}),
        how='left',
        left_on=['Element ID', 'Scale ID', 'Category'],
        right_on=['Element ID', 'Scale ID', 'Category Code']
    )

    # --- 3. Process Each Occupation and Append to its File ---
    print(f"Enriching {len(df_occupations)} files with new data...")
    for index, row in df_occupations.iterrows():
        soc_code = row['O*NET-SOC Code']
        title = row['Title']

        # Recreate the exact filename
        sanitized_title = sanitize_filename(title)
        output_filename = f"{sanitized_title}_{soc_code}.txt"
        output_filepath = os.path.join(output_path, output_filename)

        if not os.path.exists(output_filepath):
            continue

        # --- 4. Determine the Most Relevant Work Context for the Job ---
        job_context = df_merged_context[df_merged_context['O*NET-SOC Code'] == soc_code]
        context_content = ["\n\n# WORK CONTEXT"]

        if not job_context.empty:
            # Group by each specific context element (e.g., "Public Speaking")
            for element_name, group in job_context.groupby('Element Name_x'):
                # Find the category with the highest percentage score ('Data Value')
                most_relevant = group.loc[group['Data Value'].idxmax()]
                description = most_relevant['Category Description']
                context_content.append(f"- {element_name}: {description}")
        else:
            context_content.append("- No specific work context listed.")

        # --- 5. Determine the Top Interests for the Job ---
        job_interests = df_interests[df_interests['O*NET-SOC Code'] == soc_code]
        interests_content = ["\n\n# INTERESTS (RIASEC)"]

        if not job_interests.empty:
            # Map numbers to RIASEC letters/names
            riasec_map = {
                1: "Realistic",
                2: "Investigative",
                3: "Artistic",
                4: "Social",
                5: "Enterprising",
                6: "Conventional"
            }

            # Get First, Second, Third Interest High-Points if they exist
            first_hp = job_interests[job_interests['Element ID'] == '1.B.1.g']
            second_hp = job_interests[job_interests['Element ID'] == '1.B.1.h']
            third_hp = job_interests[job_interests['Element ID'] == '1.B.1.i']

            labels = ["Primary Interest", "Secondary Interest", "Tertiary Interest"]
            high_points = [first_hp, second_hp, third_hp]

            for label, hp_df in zip(labels, high_points):
                if not hp_df.empty:
                    interest_num = int(hp_df.iloc[0]['Data Value'])
                    interest_name = riasec_map.get(interest_num, f"Unknown ({interest_num})")
                    interests_content.append(f"- {label}: {interest_name}")
                else:
                    interests_content.append(f"- {label}: Not available")
        else:
            interests_content.append("- No specific interests listed.")

        # --- 6. Append All New Content to the Existing File ---
        with open(output_filepath, 'a', encoding='utf-8') as f:
            f.write('\n'.join(context_content))
            f.write('\n'.join(interests_content))

    print("Successfully added Work Context and Interests data to all files.")


In [13]:
add_context_and_interests(onet_db_path, output_path)


Starting to add Work Context and Interests data...
Loading context and interests data...
Data loaded.
Enriching 1016 files with new data...
Successfully added Work Context and Interests data to all files.


In [14]:
def add_related_occupations(onet_db_path, output_path):
    """
    Appends Related Occupations data to the existing occupation .txt files.

    Args:
        onet_db_path (str): The path to the directory containing O*NET .xlsx files.
        output_path (str): The directory where the .txt files are located.
    """
    print("\nStarting to add Related Occupations data...")

    # --- 1. Load Data ---
    try:
        occupation_data_file = os.path.join(onet_db_path, "Occupation Data.xlsx")
        related_occ_file = os.path.join(onet_db_path, "Related Occupations.xlsx")

        print("Loading related occupations data...")
        df_occupations = pd.read_excel(occupation_data_file)
        df_related = pd.read_excel(related_occ_file)
        print("Data loaded.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Could not find a required Excel file.")
        return
        
    # Create a quick lookup dictionary (hash map) for SOC Code -> Title
    # This is much faster than searching the DataFrame in every loop iteration
    soc_to_title_map = pd.Series(df_occupations.Title.values, index=df_occupations['O*NET-SOC Code']).to_dict()

    # --- 2. Process Each Occupation ---
    print(f"Enriching {len(df_occupations)} files with related occupations...")
    for index, row in df_occupations.iterrows():
        soc_code = row['O*NET-SOC Code']
        title = row['Title']

        # Recreate the filename
        sanitized_title = sanitize_filename(title)
        output_filename = f"{sanitized_title}_{soc_code}.txt"
        output_filepath = os.path.join(output_path, output_filename)

        if not os.path.exists(output_filepath):
            continue

        # --- 3. Filter and Format Related Occupations Data ---
        job_related_occs = df_related[df_related['O*NET-SOC Code'] == soc_code]
        related_content = ["\n\n# RELATED OCCUPATIONS"]

        if not job_related_occs.empty:
            for _, related_row in job_related_occs.iterrows():
                related_soc = related_row['Related O*NET-SOC Code']
                # Use the fast lookup map to get the title
                related_title = soc_to_title_map.get(related_soc, "Unknown Title")
                related_content.append(f"- {related_title}")
        else:
            related_content.append("- No related occupations listed.")

        # --- 4. Append to File ---
        with open(output_filepath, 'a', encoding='utf-8') as f:
            f.write('\n'.join(related_content))

    print("Successfully added Related Occupations data to all files.")


In [15]:
add_related_occupations(onet_db_path, output_path)


Starting to add Related Occupations data...
Loading related occupations data...
Data loaded.
Enriching 1016 files with related occupations...
Successfully added Related Occupations data to all files.
